## 무신사 크롤링

In [ ]:
from urllib.request import urlretrieve
import requests
from bs4 import BeautifulSoup
url = "https://search.musinsa.com/ranking/best?u_cat_cd=002"

In [ ]:
html_page = requests.get(url).content
soup = BeautifulSoup(html_page, 'html.parser')
soup

<!DOCTYPE html>

<html lang="ko">
<head>
<title>상품 랭킹 | 무신사 스토어</title>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
            new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
        j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
        'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-TXDSFSF');</script>
<!-- End Google Tag Manager -->
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="무신사 랭킹은 상품 매출, 판매 수량, 상품 조회 수, 작성 후기 수를 반영한 공식에 의해 선정됩니다.
무신사 스토어는 광고 목적으로 랭킹을 절대 임의 조작하지 않으므로 믿고 구매하셔도 됩니다." name="description"/>
<script type="text/javascript">
            var THIS_PAGE_GF = 'A';
        </script>
<link href="https://image.msscdn.net/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<meta content="648837775133366" id="fbAddId" property="fb:app_id"/>
<meta 

In [ ]:
products = soup.select("#goodsRankList > li") # goodsRankList id로 자식 li 모두 선택
products

[<li class="li_box" data-goods-no="2081554">
 <p class="n-label label-default txt_num_rank">
             1위
             
         </p>
 <div class="box-icon-right">
 <span class="icon-box-limited icon-reverse">한정 판매</span>
 </div>
 <div class="icon_group">
 <ul>
 <li class="icon_man sight_out" title="남성">남성</li>
 </ul>
 </div>
 <div class="li_inner">
 <div class="list_img">
 <a class="img-block" href="https://store.musinsa.com/app/goods/2081554?loc=goods_rank">
 <img alt="노스페이스(THE NORTH FACE) NJ1DM62A 남성 1996 에코 눕시 자켓" class="lazyload lazy" data-original="https://image.msscdn.net/images/goods_img/20210823/2081554/2081554_2_125.jpg" src="https://image.msscdn.net/0.gif"/>
 </a>
 </div>
 <div class="article_info">
 <p class="item_title">
 <a href="https://www.musinsa.com/brands/thenorthface">노스페이스</a>
 </p>
 <p class="list_info">
 <a href="https://store.musinsa.com/app/goods/2081554?loc=goods_rank" title="NJ1DM62A 남성 1996 에코 눕시 자켓">
                         
                         
 

In [ ]:
len(products)

90

In [ ]:
products[0]['data-goods-no']

'2081554'

In [ ]:
# 이미지 저장
img_url = products[0].select_one("img.lazyload")['data-original']
urlretrieve(img_url, img_url.split("/")[-1])

('2081554_2_125.jpg', <http.client.HTTPMessage at 0x2d59fcb21c0>)

In [ ]:
# 브랜드
item_title = products[0].select_one("p.item_title").text
print(item_title)


노스페이스



In [ ]:
# 상품명
product_name = products[0].select_one("p.list_info").text.strip()
print(product_name)

NJ1DM62A 남성 1996 에코 눕시 자켓


In [ ]:
# 가격
product_price = products[1].select_one("p.price").text.split()
product_price = product_price[-1]
print(float(product_price.replace(",", "").replace("원","")))

139900.0


In [ ]:
# 별점 등록 수
star_count = products[0].select_one("p.point > span.count").text
print(int(star_count.replace(",", "")))

1868


In [ ]:
# # 좋아요 수
# like_count = products[0].select_one("p.txt_cnt_like")
# print(like_count)

In [ ]:
product_idx = []
product_img_url_list = []
product_detail_url_list = []
product_brand_list = []
product_name_list = []
product_price_list = []
product_star_count_list = []

for product in products:
    img_url = product.select_one("img.lazyload").get("data-original")
    urlretrieve(img_url, img_url.split("/")[-1])

    # 브랜드
    product_brand = product.select_one("p.item_title").text
    
    product_detail_url = product.select_one("p.list_info > a").get("href")
    
    # 상품명
    product_name = product.select_one("p.list_info").text.strip()

    # 가격
    product_price = product.select_one("p.price").text.split()
    product_price = float(product_price[-1].replace(",", "").replace("원", ""))

    # 별점 등록 수. 별점이 있는 상품이 있고, 없는 상품이 있다.
    start_count_elem = product.select_one("p.point > span.count")
    
    # 별점이 있으면 가져오고
    if start_count_elem:
        star_count = int(start_count_elem.text.replace(",", ""))
    else:
        # 별점이 없으면 0으로 설정
        star_count = 0

    # 상품 id
    product_id = product['data-goods-no']
        
    product_idx.append(product_id)
    product_img_url_list.append(img_url)
    product_detail_url_list.append(product_detail_url.strip())
    product_brand_list.append(product_brand.strip())
    product_name_list.append(product_name.strip())
    product_price_list.append(product_price)
    product_star_count_list.append(star_count)

In [ ]:
import pandas as pd
datas = {
    "상품ID" : product_idx,
    "상품이미지URL": product_img_url_list,
    "상품상세URL": product_detail_url_list,
    "상품브랜드": product_brand_list,
    "상품명": product_name_list,
    "상품가격": product_price_list,
    "상품별점등록수": product_star_count_list
}

df = pd.DataFrame(datas)
df.head()

,상품ID,상품이미지URL,상품상세URL,상품브랜드,상품명,상품가격,상품별점등록수
0,2081554,https://image.msscdn.net/images/goods_img/2021...,https://store.musinsa.com/app/goods/2081554?lo...,노스페이스,NJ1DM62A 남성 1996 에코 눕시 자켓,299000.0,1868
1,865862,https://image.msscdn.net/images/goods_img/2018...,https://store.musinsa.com/app/goods/865862?loc...,무신사 스탠다드,캐시미어 블렌드 오버사이즈 싱글 코트 [블랙],139900.0,16084
2,1535337,https://image.msscdn.net/images/goods_img/2020...,https://store.musinsa.com/app/goods/1535337?lo...,코드그라피,[PRAUDEN] 유틸리티 덕다운 푸파 숏패딩_쿨그레이,124000.0,4286
3,2059072,https://image.msscdn.net/images/goods_img/2021...,https://store.musinsa.com/app/goods/2059072?lo...,케이투,씬에어(THINAIR) ECO LONG 다운 자켓 미드나잇 블랙,335000.0,265
4,2155848,https://image.msscdn.net/images/goods_img/2021...,https://store.musinsa.com/app/goods/2155848?lo...,노스페이스,NJ1DM63C 남성 1992 눕시 자켓,349000.0,31
